In [ ]:
import matplotlib.pyplot as pp
import pandas as pd
import seaborn as sb

from data import Movie, Rating

%config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [ ]:
movies = Movie.load()
ratings = Rating.load()

In [ ]:
ratings.value_counts()

In [ ]:
counts = ratings.data.groupby('userId')['movieId'].agg(['count']).reset_index() \
                     .groupby('count')['userId'].agg('count')

sb.countplot(counts)
pp.xlabel('Number of movies rated')
pp.ylabel('Number of users');